# <center>L2 Computational Physics</center>
---
## Week 4: Differential Equations II

In this notebook, you will write a program to study the trajectory of a spherical, iron cannonball in the presence of gravity and drag forces. 

In [1]:
import numpy
from matplotlib import pyplot as plt

In [2]:
# Define all constants to be used in the simulation
r_cb = 0.15  # Radius of cannonball in m
rho_iron = 7874  # Density of iron in kg/m^3
g = 9.81  # Acceleration due to gravity in m/s^2
kappa = 0.47  # Drag coefficient of a sphere
rho_air = 1.23  # Density of air in kg/m^3
v0 = 125.00  # Initial speed in m/s

From this information, calculate the following two variables:

In [3]:
def get_area(r):
    return numpy.pi * (r_cb**2);

def get_mass(r):
    return rho_iron * ((4/3)*numpy.pi*(r_cb**3));

area_cb = get_area(r_cb)
mass_cb = get_mass(r_cb)

In [4]:
# these tests are worth one mark
assert numpy.isclose(area_cb, 0.07068583470577035)
assert numpy.isclose(mass_cb, 111.31605249464711)

## The Differential Equations

Create a function `f` which calculates the right-hand side of the differential equation

$$ \frac{d}{dt}\left(\begin{array}{c}x\\y\\v_x \\v_y \end{array}\right) = f(x, y, v_x, v_y, t)$$

such that the differential equation represents the cannonball's motion as it moves through the air. 

- *Your function should accept two arguments:*
    - `r` *- a numpy array representing the cannonball's state, $(x, y, v_{x}, v_{y})$.*
    - `t` *- Time * 
    
- *The drag force experienced by the cannonball is given by $$\vec{F}_{\textrm{d}} = - \frac{1}{2} C_{\textrm{d}} \rho |\vec{v}|^{2} A \widehat{v}~,$$ where $\widehat{v}$ is a unit vector pointing in the direction of the cannonball's instantaneous velocity.*

In [5]:
def f(r, t):
    '''Implements differential equation for cannonball from state vector r and time t'''
    
    # Unpack array of the state
    x, y, vx, vy = r
    
    # these variables should updated in your code to be the derivatives of 
    # the x, y positions and the derivative of the x, y velocities. 
    dx_dt, dy_dt, dvx_dt, dvy_dt = 0, 0, 0, 0
    
    drag_force = (1/2)*kappa*area_cb*rho_air*(vx**2 + vy**2)
    
    dvy_dt = -g - (drag_force)/mass_cb *((vy)/(numpy.sqrt(vx**2+vy**2)))
    dvx_dt = - ((drag_force)/mass_cb) * ((vx)/(numpy.sqrt(vx**2+vy**2)))
    dy_dt = (dvy_dt * t) + vy
    dx_dt = (dvx_dt *t) + vx
    return numpy.array([dx_dt, dy_dt, dvx_dt, dvy_dt])

Check your result:

In [6]:
r_test = [0, 0, 88.38834764831843, 88.38834764831843]
assert len(f(r_test, 0))== 4

In [7]:
r_test = [0, 0, 88.38834764831843, 88.38834764831843]
assert numpy.isclose(f(r_test, 0), [ 88.38834765,  88.38834765,  -2.0279283 , -11.8379283 ]).all()

## Solving the Differential Equations

Create the function `solve_euler(state_initial, t1, n_panels)` which solves the differential equations using Euler's method.

- `state_initial` *is a numpy array containing the initial conditions $(x_{0}, y_{0}, v_{x0}, v_{y0})$*
- *When working with a numpy array, you can use the same commands as those used for single variables - for instance,* `state += f(state, t)`.
- *This function should record all state values at each timepoint in the variable `history` and return it. This will be similar to last week, only this time you will have to populate a 2D array.* 


In [30]:
def solve_euler(state_initial, t1, n_steps):
    t = 0
    '''Solves ODE using Euler's method from state_initial to end time t1 using n_panels panels'''
    # Define array for trajectory history
    history = numpy.empty((n_steps+1,4))  # Number of columns is equal to number of variables to solve for (4)
    history[0] = state_initial
    # you should now populate the history array
    # YOUR CODE HERE
    dt = t1 / n_steps
    for i in range (1, n_steps):
        a = f(state_initial, t)
        print (a)
        ay = a[3]
        ax = a[2]
        vy = ay * t + state_initial[3]
        vx = ax * t + state_initial[2]
        y = ay * (t**2/2) + state_initial[3]*t + state_initial[1]
        x = ax * (t**2/2) + state_initial[2]*t + state_initial[0]
        state_initial = numpy.array([x,y,vx,vy])
        history[i] = state_initial
        print(state_initial)
        t += dt
    return history
r_test = [0, 0, 88.38834764831843, 88.38834764831843]
solve_euler(r_test, 25.0, 10)

[ 88.38834765  88.38834765  -2.0279283  -11.8379283 ]
[ 0.          0.         88.38834765 88.38834765]
[ 83.3185269  58.7935269  -2.0279283 -11.8379283]
[214.63359319 183.97734319  83.3185269   58.7935269 ]
[ 75.52116386   4.24133561  -1.55947261 -10.91043826]
[611.7328201  341.56449945  75.52116386   4.24133561]
[ 67.65740851 -69.77529988  -1.04850071  -9.86888473]
[1148.65246649   95.81213343   67.65740851  -69.77529988]
[  55.5878997  -155.42797718   -1.20695088   -8.56526773]
[ 1764.87900755 -1030.20425187    55.5878997   -155.42797718]
[  34.53537767 -219.18853129   -1.68420176   -5.10084433]
[ 2328.14949111 -3371.5574298     34.53537767  -219.18853129]
[  13.43716037 -232.43273453   -1.40654782   -0.88294688]
[ 2687.94352636 -6758.7169234     13.43716037  -232.43273453]
[ 3.38834359e+00 -2.30285744e+02 -5.74218101e-01  1.22685173e-01]
[ 2.83516669e+03 -1.08075036e+04  3.38834359e+00 -2.30285744e+02]
[ 5.23644138e-01 -2.31789006e+02 -1.43234973e-01 -7.51630910e-02]
[ 2.87428656e+

array([[ 0.00000000e+000,  0.00000000e+000,  8.83883476e+001,
         8.83883476e+001],
       [ 0.00000000e+000,  0.00000000e+000,  8.83883476e+001,
         8.83883476e+001],
       [ 2.14633593e+002,  1.83977343e+002,  8.33185269e+001,
         5.87935269e+001],
       [ 6.11732820e+002,  3.41564499e+002,  7.55211639e+001,
         4.24133561e+000],
       [ 1.14865247e+003,  9.58121334e+001,  6.76574085e+001,
        -6.97752999e+001],
       [ 1.76487901e+003, -1.03020425e+003,  5.55878997e+001,
        -1.55427977e+002],
       [ 2.32814949e+003, -3.37155743e+003,  3.45353777e+001,
        -2.19188531e+002],
       [ 2.68794353e+003, -6.75871692e+003,  1.34371604e+001,
        -2.32432735e+002],
       [ 2.83516669e+003, -1.08075036e+004,  3.38834359e+000,
        -2.30285744e+002],
       [ 2.87428656e+003, -1.54282511e+004,  5.23644138e-001,
        -2.31789006e+002],
       [ 1.43839488e-279,  1.09385332e-303,  1.43373535e-298,
         8.87200301e-317]])

Test your solution:

In [25]:
# this test is worth 4 marks
r_test = [0, 0, 88.38834764831843, 88.38834764831843]
assert numpy.isclose(solve_euler(r_test, 25.0, 10), numpy.array(
    [[   0.        ,    0.        ,   88.38834765,   88.38834765],
     [ 220.97086912,  220.97086912,   83.3185269 ,   58.7935269 ],
     [ 429.26718637,  367.95468637,   79.41984538,   31.51743125],
     [ 627.81679983,  446.7482645 ,   76.30595276,    5.75669607],
     [ 818.58168171,  461.14000469,   73.62655732,  -18.97044366],
     [1002.64807501,  413.71389553,   71.05785323,  -42.83359748],
     [1180.2927081 ,  306.62990184,   68.35253688,  -65.72783569],
     [1351.1740503 ,  142.31031262,   65.37829955,  -87.3928075 ],
     [1514.61979917,  -76.17170612,   62.10406078, -107.54105068],
     [1669.87995113, -345.02433282,   58.56508321, -125.93786257],
     [1816.29265915, -659.86898925,   54.83262575, -142.43661744]])).all()

[ 88.38834765  88.38834765  -2.0279283  -11.8379283 ]
2.5
[ 83.3185269  58.7935269  -2.0279283 -11.8379283]
5.0
[ 75.52116386   4.24133561  -1.55947261 -10.91043826]
7.5
[ 67.65740851 -69.77529988  -1.04850071  -9.86888473]
10.0
[  55.5878997  -155.42797718   -1.20695088   -8.56526773]
12.5
[  34.53537767 -219.18853129   -1.68420176   -5.10084433]
15.0
[  13.43716037 -232.43273453   -1.40654782   -0.88294688]
17.5
[ 3.38834359e+00 -2.30285744e+02 -5.74218101e-01  1.22685173e-01]
20.0
[ 5.23644138e-01 -2.31789006e+02 -1.43234973e-01 -7.51630910e-02]
22.5


AssertionError: 

## Finding the range

To find the range of the projectile we will look for the $x$ coordinate when the trajectory crosses the $y=0$ line. In most cases that point will not be one of the steps but will be between two steps. We will use a linear approximation to determine this point given the last point with a positive $y$ value and the first point with a negative $y$ value. Implement the function `find_zero_linear` that takes as argument the two values of $x$ `x1` and `x2` and the heights `y1` and `y2` and returns the value of $x$ at which the line between $(x_1,y_1)$ and $x_2,y_2$ crosses the $y=0$ line.  

In [ ]:
def find_zero_linear(x1, x2, y1, y2):
    if y1*y2 > 0:
        print("I expect y1 and y2 to have opposite signs!")
    # YOUR CODE HERE

In [ ]:
# these tests are worth 2 marks
assert numpy.isclose(find_zero_linear(-1,1,3,-3),0.0)
assert numpy.isclose(find_zero_linear(-1,1,1,-3), -0.5)

Given the function above we can define a function to determine the range. (Don't modify it!)

In [ ]:
def find_range(history):
    all_xs = history[:,0]
    all_ys = history[:,1]
    negatives = numpy.argwhere(all_ys<0)
    if len(negatives) == 0 :
        print ("The projectile did not touch down! Returning the last known location")
        return all_xs[-1]
    (index,) = negatives[0]
    y1, y2 = all_ys[index-1], all_ys[index]
    x1, x2 = all_xs[index -1], all_xs[index]
    return find_zero_linear(x1,x2,y1,y2)

In [ ]:
r_test = [0, 0, 88.38834764831843, 88.38834764831843]
h = solve_euler(r_test, 25.0, 10000)
assert numpy.isclose(find_range(h),1306.2541)

## Plotting task

Create a plot to show the trajectories for different values of the initial angle. Use the same velocity $v0=125\rm{m/s}$ for all angles. The plot should have axis labels and a title. [4 marks]


In [ ]:
n_steps = 1000
thetas = range(5, 90, 5) # the angles in degrees

initial_conditions = [0, 0, 88.38834764831843, 88.38834764831843]    
values_euler = solve_euler(initial_conditions, 300, n_steps)
xs_euler, ys_euler = values_euler[:,0], values_euler[:,1]
plt.plot(xs_euler, ys_euler, color='blue', linestyle='--')
plt.xlim(0,1500)
plt.ylim(0,800);

Create a plot to show the range for different values of the initial velocity. Use the same angle $\theta=60^\circ$ for all velocities. The plot should have axis labels and a title and legend. Produce one curve with and one curve without the effect of air resistance. [5 marks]


In [ ]:
n_steps = 1000
max_time = 300
v0s = numpy.linspace(50, 1000, 20)

ranges = []
ranges_noresistance = []
theta = numpy.deg2rad(60)

# Your code here
